# Understanding molecular bases of drug response and drug synergy
Understanding synergistic effects of drugs is key to develop effective intervention strategies targeting diseases (such as AD or T2D or both) and provides unprecedented opportunities to repurpose existing drugs. The AstraZeneca-Sanger Drug Combination Prediction DREAM Challenge provides a rich data source aiming to understand the synergistic drug behavior based on pretreatment data and spans cell viability data over 118 drugs and 85 cancer cell lines (primarily colon, lung, and breast). In collaborating with Dr. Baldo Oliva's group at GRIB, UPF-IMIM, we have been working on identifying the effects of confounding factors in the data set such as dosage and genetic background of the cell lines and developing algorithms that can predict the individual and synergistic effects of drugs. 

The challenge has two subtasks: predicting drug synergy *(i)* using mono synergy data *(ii)* without using mono synergy data. The participants are free to use any other data source (such as cell line gene expression, gene mutation, drug target information provided in the challenge or external data sets) and submit their predictions in 3-4 rounds. The other task of the challenge requires making predictions for drug combinations and cell lines for which no previous training data available (making it hard to build a machine learning predictor). 

Check challenge info and timelines at https://www.synapse.org/#!Synapse:syn4231880/wiki/235652

Next deadline (for both tasks): January, 26th 2015

<For the first round of the challenge, we have build machine learning models to predict the synergy of drugs for both of these tasks and choice the best performing models to submit predictions. Among various machine learning models, we found a combination of bootstrapped and ensemble tree-based predictors achieved best performance on the training data set for. 

To improve the prediction performance we have incorporated mutation data (of drug targets in a given cell line) and interactome based contribution of the drug combination compared to the effect of drugs separately. To assess interactome based contribution of a drug or combination (characterized by a set of targets), we have used GUILD, a network-based functional prioritization tool. 

Interestingly, using GUILD, only the predictions for subtask *(ii)* improved but not for subtask *(i)*. We suspect this is due to the mono therapy response data describing the synergy best and addition of new features (such as the ones based on expression, mutation, interactome) potentially causing the predictor to overfit to the training data set.>

## Data overview

Challenge training data consists of 2199 samples providing information on 169 drug pairs over 85 cell lines.

In [42]:
source("dream.R")
parameters = initialize()
dat = overview(parameters, summarize=F)

[1] "Number of samples: 2199"
[1] "Number of drug pairs: 169, number of cell lines: 85"
[1] "Features: CELL_LINE, COMPOUND_A, COMPOUND_B, MAX_CONC_A, MAX_CONC_B, IC50_A, H_A, Einf_A, IC50_B, H_B, Einf_B, SYNERGY_SCORE, QA, COMBINATION_ID"


## Data cleaning and imputation

- Filter samples with low quality (404 samples):
$$QA < 1$$

- Filter samples with low sensitivity (3 cell lines) based on the observation that higher Einf correlates with lower synergy. Define Einf of the drug pair A,B as follows:
$$min((Einf_A + Einf_B) / 2)$$ 

- Filter correlated features (None):
$$ PCC > 0.75 $$

Both min (-588.221) and max (6737.175) synergy instances have low quality. After filtering synergy scores range between -179 and 237.

- Impute missing values using k-nearest-neighbor ($k = 5$)

In [43]:
dat = filter(dat, cutoff=40)

[1] "Correlated features:"
[1] "Correlation between einf.min and syn.med: -0.235249 0.031231"
[1] "Insensitive cell lines: 22RV1, KU-19-19, VCaP"
[1] "Number of samples with QA < 1: 404, Einf > 40: 7"


## Feature definition and prediction models

***Baseline prediction***

- Monotherapy response based
    * max concentration
    * viability at max kill
    * IC50 
    * slope of the fit to the dose response curve
    
***Expression based***

- The average gene expression of the targets of two drugs in the cell line
    * $gexp = abs(gexpA - gexpB)$, where for each cell line where gexpT is
    
$$ gexp(T, cell) = \frac{1}{||T||} * \sum_{t \in targets(T)} E(t, cell) $$ where $E$ is the gene expression matrix, $T$ is the drug tested in a given $cell$ line. Expression values are converted to z-scores and absolute value of the z-score is used. 

***Mutation based***

- The average mutation score of the targets of the two drugs in the cell line
    * $mut = abs(mutA - mutB)$ , where for each cell line where mutT is
    
$$ mut(T, cell) = \frac{1}{||T||} * \sum_{t \in targets(T)} M(t, cell) $$ where $M$ is the mutation, $T$ is the drug tested in a given $cell$ line. Genes are assigned mutation score based on the "Description" field in the annotation file (0 if the mutation is silent or of unknown impact, 2 if the mutation is associated to cancer with respect to FATHMM prediction and 1 otherwise). Impute missing values using k-nearest-neighbor ($k = 5$).

***GUILD based***

- The network-impact score distribution of the genes in the overlap between top 500 genes in GUILD-based prioritization of drug targets of A and B, respectively. 
    * guild.med (median of the distribution of the network impact)
    * guild.max 

The network-impact is calculated as
$$ impact(A,B) = GUILD({A,B}) - (GUILD(A) + GUILD(B)) / 2 $$ where $GUILD(X)$, is the GUILD scores of the genes when genes in X are used as seeds. Top scoring 500 genes common in $GUILD(A)$ and $GUILD(B)$ are considered to calculate the impact score distribution.
    
***Drug similarity based***

- If the drugs are similar, the effect is expected to be synergistic (i.e. Loewe additivity)
    * sim.target: common targets
        $$ sim(A, B) = \frac{T(A) \cap T(B)}{T(A) \cup T(B)}  $$
        
    * sim.chemical: chemical formula similarity, calculated using Tanimota similarity coefficient (Jaccard index of molecular fingerprints)

***KEGG pathways***

- Cancer related from  KEGG pathways. These pathways are "pathways in cancer", "aminoacyl-tRNA biosynthesis", "MAPK signaling pathway", "NF-kappa B signaling pathway". For genes in these pathways,
    * involvement of drug targets in these pathways (kegg.in, 2: targets of both drugs in combination are in the pathway, 1: only targets of one are in the pathway, 0: none of the targets are in the pathway)
    * gene expression (kegg.gexp.med and kegg.gexp.max: the median and max of the distribution)
    * mutation (kegg.mut.med and kegg.mut.max)
    * CNV (kegg.cnv.med and kegg.cnv.max)
    
***Cancer genes***

- COSMIC genes from http://cancer.sanger.ac.uk/census/ (572 genes).
    * involvement of drug targets in these pathways (cosmic.in)
    * gene expression (cosmic.gexp.med and cosmic.gexp.max)
    * mutation (cosmic.mut.med and cosmic.mut.max)
    * CNV (cosmic.cnv.med and cosmic.cnv.max)

***Combined***

- The best performing combination of features using RandomForest and Generalized Boosted Regression Models:
    + gexp
    + mut
    + cnv
    + guild.med
    + guild.max
    + sim.target
    + sim.chemical
    + kegg.in
    + cosmic.in

In [44]:
results(parameters)

,features,correlation
1,"gexp, mut",19.7
2,"gexp, mut, cnv",28.2
3,"gexp, mut, cnv, guild",35.2
4,"gexp, mut, cnv, guild(2), sim(2)",41.3
5,"gexp, mut, cnv, guild(2), sim(2), kegg.in, cosmic.in",44.9


## Final predictors and confidence assignment

>***Predictor for challenge 1 subtask 1*** 
The best performing predictor using the response data and the features above achieves an accuracy (assessed by correlation between predicted and observed synergy scores) of 0.41 in the training set. Note that this value fluctuates depending on the folds used in cross validation (+/- 0.10). 

>***Predictor for challenge 1 subtask 2*** 
Achieves an accuracy (assessed by correlation between predicted and observed synergy scores) of 0.45
    
>***Predictor for challenge 2*** 
This challenge requires predicting drug combinations and cell lines for which no previous training data is available, thus makes it harder to find features that would work over all the test data (due to the missing values). The predictor achieves a correlation of 0.29

>***Confidence assignment***
We observed that the predictions tend to fail for higher synergy scores, accordingly we defined the following confidence scoring:
$$confidence = 1 - abs(synergy) / max(abs(synergy))$$

## DREAM evaluation 

>The global correlation values from DREAM (assessed by real values in the test set) are substantially lower than the correlation values in the training set (assessed by model development on 70% of the training data using cross validation and validation using 30% of the data).
* Challenge 1 subtask 1: 0.13 (global), 0.19 (mean of top 10-20-30%), 
* Challenge 1 subtask 2: 0.09 (global), 0.03 (mean of top 10-20-30%)
* Challenge 2: TBA
* Overall ranking: ~50% of all submissions

## TODO

- Update the categorical prediction for challenge 2, need to balance the training set (0: 80%, 1: 20%)
- Check scoring evaluation details announced by DREAM
- Consider confidence scoring based on the sensitivity of the cell lines
- Use several features proposed in Sun et al., 2015, Nat Comms "Combining genomic and network characteristics for extended capability in predicting synergistic drugs for cancer" http://www.nature.com/ncomms/2015/150928/ncomms9481/full/ncomms9481.html
    + MI GO BP
    + Unrelated pathway ratio
    + distance between targets in PPI
    + degree & centrality in PPI
- Incorporate external data / synergy modeling